# Метод штрафов. ADMM.

## Вступление

Ранее мы частенько работали с ERM на основе логистической регрессии, однако, в данной домашней работе к рассмотрению будет предложена функция, лежащая в основне SVM (чисто ради разнообразия). Ставится следующая задача оптимизации:
$$
\begin{equation}
\min \limits_{w \in \mathbb{R}^d} \frac{1}{2} \| w \|^2 + C \sum \limits_{i = 1}^n \max \left(1 - y_i \cdot \langle X_i, w \rangle , 0 \right).
\end{equation}
$$
Член с суммой есть ни что иное, как *hinge_loss*, довольно популярная функция потерь при обучении классификаторов.

## Основная часть (10 баллов)

__a) (1 балл)__ Переформулируйте задачу (1) в задачу оптимизации с ограничениями. *Подсказка*: попробуйте представить слагаемое в сумме как $s_i = (Bw)_i + e_i$, где $B \in \mathbb{R}^{n \times d}$ -- некоторая матрица, а $e \in \mathbb{R}^n$ -- вектор, составленный из единиц. Для удобства обозначений также замените суммирование на функцию
$$
p(s) = \sum \limits_{i = 1}^n \max(s_i, 0).
$$

In [ ]:
# Ваше решение (Markdown)

__б) (1.5 балла)__ Для полученной задачи оптимизации с ограничениями напишите функцию Лагранжа $L(w, s, \lambda)$. В качестве ограничений можно взять функцию $s$.

In [ ]:
# Ваше решение (Markdown)

__в) (0.5 балл)__ Запишите расширенный Лагранжиан задачи $L_{\rho}(w, s, \lambda)$, введя параметр $\rho$.

In [ ]:
# Ваше решение (Markdown)

__г) (2 балла)__ Как вы помните из лекции, на шаге алгоритма ADMM мы решаем задачу минимизации расширенного Лагранжиана по параметру $s$:
$$
\varphi (w) = \min \limits_{s} L_{\rho} (w, s, \lambda).
$$
Переформулируйте задача минимизации так, чтобы члены с $w$ и $\lambda$ были вытащены из под $\min$. То есть вид должен быть такой:
$$
\varphi (w) = c_1 \|w\|^2 + c_2 \| \lambda\|^2 + c_3 \min \limits_{s} \left[\|s - t(w)\|^2 + c_4 p(s) \right].
$$
Здесь $t(w)$ -- функция, которая зависит только от переменной $w$.

In [ ]:
# Ваше решение (Markdown)

__д) (2 балла)__ Вычислите градиент $\nabla \varphi (w)$ аналитически. Считайте, что в точке оптимума значение $s^* = s^*(w^*) = \text{Prox}_p^{\rho^{-1}} (t(w^*))$. Воспользовавшись знанием с предыдущих лекций, покажите, что 
$$
\left[ \text{Prox}_p^{\rho^{-1}}\left(z \right) \right]_i = \begin{cases} z_i - \frac{C}{\rho}, & z_i > \frac{C}{\rho},\\ 0, & 0 \leq z_i \leq \frac{C}{\rho}, \\ z_i, & z_i < 0. \end{cases}
$$

In [ ]:
# Ваше решение (Markdown)

__е) (3 балла)__ Напомним основные итерации ADMM на $k$-ой итерации:
1. Вычислить новые значения $w^{k + 1}$ и $s^{k + 1}$ при решении задачи минимизации $\min \varphi (w)$:
$$w^* = \arg\min \varphi (w); \qquad s^* = s^*(w^*) = \text{Prox}_p^{\rho^{-1}} (t(w^*))$$
2. Обновить параметр $\lambda^{k + 1}$ по теории (_покажите это!_):
   $$\lambda^{k + 1} = \lambda^{k} - \rho (s - Bw - e).$$ 

Реализуйте метод ```ADMM``` для решения задачи оптимизации ```hinge_loss```, проверьте сходимости при разных значениях коэффициента $C$. Рассмотрите следующие значения: [1e-10, 1e-8, 1e-6]. Параметр $\rho$ положите равным 1. Критерием останова будет являться норма градиента функции $\phi$ в новой точке.

In [ ]:
# Ваше решение (Markdown + Code)

In [ ]:
dataset = "mushrooms.txt" 

from sklearn.datasets import load_svmlight_file
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]

#(преобразуйте метки к формату [-1, +1])
y = ...

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=57)

## Дополнительная часть (всего 10 баллов)

__а) (3 балла)__ На лекции был использован метод первого порядка в качестве спуска по переменным. Однако, такой метод проигрывает по итеративной сложности методу Ньютона... Но функция-то в нашем случае не дважды диффиренцируема. Значит нужно рассматривать суб-гессиан. Найдите второй суб-дифференциал $\partial ^2 \varphi(w)$.

(*Указание:* возьмите честно второй раз дифференциал от формулы из пункта __д)__, используя градиент в тех местах, когда это возможно).

In [ ]:
# Ваше решение (Markdown)

__б) (7 баллов)__ Запишем теперь [полугладкий метод Ньютона](https://www.polyu.edu.hk/ama/profile/dfsun/A%20QUADRATICALLY%20CONVERGENT%20NEWTON%20METHOD%20FOR_published.pdf) для поиска аргмина. Так как мы не умеем вычислять точно гессиан оптимизируемой функции, то прибегнем к методу сопряженных градиентов для подсчета направления спуска. На $k$-ой итерации:
1. Выбрать элемент $V_k \in \partial^2 \varphi(w^k)$.
2. Выполнить процедуру сопряженного градиента для поиска аппроксимации решения $d^k$ уравнения
$$
V_k d^k + \nabla \varphi(w^k) = 0.
$$
3. При помощи линейного поиска, как в методе BFGS находим параметр $\alpha^k$ и делаем обновление переменной:
$$w^{k + 1} = w^k + \alpha_k d^k.$$

Теперь, мы наконец-то готовы все собрать в один метод ADMM. На $k$-ой итерации
1. Вычислить новые значения $w^{k + 1}$ используя полугладкий метод Ньютона.
2. Обновить $s^{k + 1} = \text{Prox}_{p}^{\rho^{-1}} (w^{k + 1})$.
3. Обновить параметр $\lambda^{k + 1}$ по теории.

Реализуйте метод ```ADMM_SemiSmoothNewton``` и постройте график зависимости критерия от итерации. Поиграйте со значениями параметров, постройте сравнительные графики.

In [ ]:
# Ваше решение (Code)

# Интересно почитать

Казалось бы, метод ADMM выглядит более громоздко, нежели привычные методы оптимизации без ограничений (SGD, NAG, DNM и другие), однако, на практике у него довольно много применений. Если вас заинтересовала возможность узнать больше о практических применения, рекоммендуется к прочтению [статья](https://web.stanford.edu/~boyd/papers/pdf/admm_distr_stats.pdf#page=64.25) от одного из столпов оптимизации Бойда. В ней рассматривается distributed подход (с множеством вычислительных единиц) к использованию метода ADMM при решении выпуклых задач оптимизации в машинном обучении и не только.   